In [1]:
import os
import numpy as np
import nibabel

from tqdm import tqdm
from data_preparation.utils import crop_non_zero_3d
import scipy.ndimage as ndimage

In [2]:
root_path = "/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation"
save_root_path = "/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Z_CROP_V2"
save_anom_root_path = "/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Z_CROP_ANOM"

sub_paths = [root_path + f"/{sub_dir}" for sub_dir in os.listdir(root_path)]

mask_str = "output/li.nii"
scan_str = "input/li_0000.nii"

all_pre_mask_path = []
all_portal_mask_path = []
all_pre_path = []
all_portal_path = []


for idx, sub_path in enumerate(sub_paths):

    if "P" in os.listdir(sub_path):
        continue
    else:
        pre_phase_path = sub_path + "/D/"
        portal_phase_path = sub_path + "/V/"

        pre_mask_path = pre_phase_path + mask_str
        pre_path = pre_phase_path + scan_str

        portal_mask_path = portal_phase_path + mask_str
        portal_path = portal_phase_path + scan_str

        all_pre_mask_path.append(pre_mask_path)
        all_portal_mask_path.append(portal_mask_path)
        all_pre_path.append(pre_path)
        all_portal_path.append(portal_path)
# sort paths
all_pre_mask_path = sorted(all_pre_mask_path)
all_portal_mask_path = sorted(all_portal_mask_path)
all_pre_path = sorted(all_pre_path)
all_portal_path = sorted(all_portal_path)

list_A = []
list_B = []
for p in range(len(all_pre_mask_path)):
    print(f"D out: {all_pre_mask_path[p]}, P out: {all_portal_mask_path[p]}")
    print(f"D in: {all_pre_path[p]}, P in: {all_portal_path[p]}")
    list_A.append(all_portal_path[p].split("/")[7])
    list_B.append(all_portal_path[p].split("/")[7])
# print(list_A)
# print(list_B)
set_A = set(list_A)
set_B = set(list_B)
print(set_A.symmetric_difference(set_B))

D out: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1104/D/output/li.nii, P out: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1104/V/output/li.nii
D in: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1104/D/input/li_0000.nii, P in: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1104/V/input/li_0000.nii
D out: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1106/D/output/li.nii, P out: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1106/V/output/li.nii
D in: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1106/D/input/li_0000.nii, P in: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1106/V/input/li_0000.nii
D out: /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom_Segmentation/HCC_1107/D/output/li.nii, P out: /med

In [3]:
def ensure_multiples_of_16(image_3d):
    """
    Ensures that the input image shape is multiple of 16
    image_3d: numpy 3d array
    """

    multiples_of_16 = [16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256, 272, 288, 304, 320,
                       336, 352, 368, 384, 400, 416, 432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624,
                       640, 656, 672, 688, 704, 720, 736, 752, 768, 784]

    # img_cropped_3d = crop_non_zero_3d(image_3d)
    img_cropped_3d = image_3d

    new_dimension = [img_cropped_3d.shape[0], img_cropped_3d.shape[1], img_cropped_3d.shape[2]]  # x, y , z

    for d in range(len(new_dimension)):
        if img_cropped_3d.shape[d] not in multiples_of_16:
            for m in multiples_of_16:
                if m > img_cropped_3d.shape[d]:
                    new_dimension[d] = m
                    break
    padding = np.array(new_dimension) - np.array(img_cropped_3d.shape)

    new_pad = []
    for pad in padding:
        if pad % 2 == 0:  # the number is even
            p1 = pad // 2
            p2 = pad // 2
            new_pad.append((p1, p2))
        else:  # the number is odd
            p1 = int((pad - 1) / 2)  # (5 - 1) / 2 = 2
            p2 = int(((pad - 1) / 2) + 1)  # (5 - 1)/ 2 => 2 + 1 = 3
            new_pad.append((p1, p2))

    padded_3d = np.pad(img_cropped_3d, new_pad, 'constant', constant_values=(-1000, -1000))
    return padded_3d

In [4]:
def match_spatial_size(image_3d_A, image_3d_B):
    # -------------------- Make sure the spatial size of images matches each other --------------------
    # move this part to data preprocessing section
    if image_3d_A.shape[0] < image_3d_B.shape[0]:
        image_3d_A = np.pad(image_3d_A, ((0, abs(image_3d_B.shape[0] - image_3d_A.shape[0])),
                                         (0, 0), (0, 0)), 'constant',  constant_values=(-1000, -1000))

    if image_3d_A.shape[0] > image_3d_B.shape[0]:
        image_3d_B = np.pad(image_3d_B, ((0, abs(image_3d_B.shape[0] - image_3d_A.shape[0])),
                                         (0, 0), (0, 0)), 'constant',  constant_values=(-1000, -1000))

    if image_3d_A.shape[1] < image_3d_B.shape[1]:
        image_3d_A = np.pad(image_3d_A, ((0, 0),
                                         (0, abs(image_3d_B.shape[1] - image_3d_A.shape[1])), (0, 0)), 'constant',  constant_values=(-1000, -1000))

    if image_3d_A.shape[1] > image_3d_B.shape[1]:
        image_3d_B = np.pad(image_3d_B, ((0, 0),
                                         (0, abs(image_3d_B.shape[1] - image_3d_A.shape[1])), (0, 0)), 'constant',  constant_values=(-1000, -1000))

    if image_3d_A.shape[2] < image_3d_B.shape[2]:
        image_3d_A = np.pad(image_3d_A, ((0, 0),
                                         (0, 0), (0, abs(image_3d_B.shape[2] - image_3d_A.shape[2]))), 'constant',  constant_values=(-1000, -1000))

    if image_3d_A.shape[2] > image_3d_B.shape[2]:
        image_3d_B = np.pad(image_3d_B, ((0, 0),
                                         (0, 0), (0, abs(image_3d_B.shape[2] - image_3d_A.shape[2]))), 'constant',  constant_values=(-1000, -1000))

    return image_3d_A, image_3d_B

In [5]:
slice_thresh = 112

for i in tqdm(range(len(all_pre_path))):

    # save dir
    pre_str = "/delayed/"
    portal_str = "/portal/"
    anom_str_d = "/anomaly_d/"
    anom_str_p = "/anomaly_p/"
    save_dir_pre = all_pre_path[i].split("/")[7]
    save_dir_portal = all_portal_path[i].split("/")[7]

    save_pre_path = save_root_path + pre_str + save_dir_pre + "_d"
    save_portal_path = save_root_path + portal_str + save_dir_portal + "_p"
    save_anom_d_path = save_anom_root_path + anom_str_d + save_dir_portal + "_anom_d"
    save_anom_p_path = save_anom_root_path + anom_str_p + save_dir_portal + "_anom_p"



    # pre part
    pre_scan = nibabel.load(all_pre_path[i])
    pre_scan_array = pre_scan.get_fdata()

    # pre masks
    pre_mask_file = nibabel.load(all_pre_mask_path[i])
    pre_mask_array = pre_mask_file.get_fdata()
    pre_mask_array[pre_mask_array == 2] = 1

    # cropped_pre_mask_array = crop_non_zero_3d(pre_mask_array)

    indices_3d_pre = np.nonzero(pre_mask_array)
    xyz_min_pre = np.min(indices_3d_pre, 1)
    xyz_max_pre = np.max(indices_3d_pre, 1)

    cropped_pre_scan= pre_scan_array[: , : , xyz_min_pre[2]:xyz_max_pre[2]]
    # pre_mask_m16 = ensure_multiples_of_16(cropped_pre_mask_array)
    pre_scan_m16 = ensure_multiples_of_16(cropped_pre_scan)

    # pre_mask_matched, pre_scan_matched = match_spatial_size(pre_mask_m16, pre_scan_m16)

    # pre_scan_new = pre_scan_matched * ndimage.binary_fill_holes(pre_mask_matched)


    # portal part
    portal_scan = nibabel.load(all_portal_path[i])
    portal_scan_array = portal_scan.get_fdata()

    # portal masks
    portal_mask_file = nibabel.load(all_portal_mask_path[i])
    portal_mask_array = portal_mask_file.get_fdata()
    portal_mask_array[portal_mask_array == 2] = 1

    # cropped_portal_mask_array = crop_non_zero_3d(portal_mask_array)

    indices_3d_portal = np.nonzero(portal_mask_array)
    xyz_min_portal = np.min(indices_3d_portal, 1)
    xyz_max_portal = np.max(indices_3d_portal, 1)

    cropped_portal_scan= portal_scan_array[: , :, xyz_min_portal[2]:xyz_max_portal[2]]

    # portal_mask_m16 = ensure_multiples_of_16(cropped_portal_mask_array)
    portal_scan_m16 = ensure_multiples_of_16(cropped_portal_scan)

    # ensure equal shape size
    # portal_mask_matched, portal_scan_matched = match_spatial_size(portal_mask_m16, portal_scan_m16)

    # portal_scan_new = portal_scan_matched * ndimage.binary_fill_holes(portal_mask_matched)

    matched_pre, matched_portal = match_spatial_size(pre_scan_m16, portal_scan_m16)

    if matched_pre.shape[2] > slice_thresh:
        print(f"matched_pre.shape[2]: {matched_pre.shape[2]}")
        os.makedirs(save_anom_d_path , exist_ok=True)
        os.makedirs(save_anom_p_path , exist_ok=True)

        save_anom_d_nifti_path = os.path.join(save_anom_d_path, f"crop_z_001.nii")
        nifti_pre_new = nibabel.Nifti1Image(matched_pre, pre_scan.affine, pre_scan.header)
        nibabel.save(nifti_pre_new, save_anom_d_nifti_path)

        save_anom_p_nifti_path = os.path.join(save_anom_p_path, f"crop_z_001.nii")
        nifti_portal_new = nibabel.Nifti1Image(matched_portal, portal_scan.affine, portal_scan.header)
        nibabel.save(nifti_portal_new, save_anom_p_nifti_path)

    else:

        os.makedirs(save_pre_path , exist_ok=True)    # pre scans
        os.makedirs(save_portal_path , exist_ok=True)    # portal scans

        save_pre_nifti_path = os.path.join(save_pre_path, f"crop_z_001.nii")
        nifti_pre_new = nibabel.Nifti1Image(matched_pre, pre_scan.affine, pre_scan.header)
        nibabel.save(nifti_pre_new, save_pre_nifti_path)

        save_portal_nifti_path = os.path.join(save_portal_path, f"crop_z_001.nii")
        nifti_portal_new = nibabel.Nifti1Image(matched_portal, portal_scan.affine, portal_scan.header)
        nibabel.save(nifti_portal_new, save_portal_nifti_path)
    # if i==4:
    #     break
print('save completed!')

  0%|          | 0/168 [00:02<?, ?it/s]


KeyboardInterrupt: 